In [ ]:
!pip install transformers qdrant-client torch

In [ ]:
!pip install FlagEmbedding

In [ ]:
! pip install faiss-cpu

In [ ]:
import subprocess

# Clone the Annoy repository
subprocess.run(["git", "clone", "https://github.com/spotify/annoy.git"])

# Install the Annoy package
subprocess.run(["pip", "install", "annoy"])


CompletedProcess(args=['pip', 'install', 'annoy'], returncode=0)

In [ ]:
import logging
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance, PointStruct
from sentence_transformers import SentenceTransformer
import numpy as np
import os

from qdrant_client import QdrantClient
from qdrant_client.http import models as rest
from FlagEmbedding import FlagModel
from qdrant_client.http.models import VectorParams, Distance
from FlagEmbedding import FlagReranker
import string
from FlagEmbedding import BGEM3FlagModel
from qdrant_client.http.models import PointStruct, VectorParams, Distance
from qdrant_client import QdrantClient
from FlagEmbedding import FlagModel
from qdrant_client.http.models import VectorParams, Distance
#from splitingdata import Chunking
import logging
from sentence_transformers import SentenceTransformer
import string
from FlagEmbedding import BGEM3FlagModel
import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import re

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
from annoy import AnnoyIndex
import faiss
import numpy as np
import random
import time
import pickle

In [ ]:

class Chunking(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def read_data(self, filename):
        df = pd.read_csv(filename)
        return df

    def fit(self, X, y=None):
        pass

    def clean_data(self, text):
        url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
        text = re.sub(url_pattern, '', text)
        emoji_pattern = re.compile(r'[\U00010000-\U0010ffff]')
        text = re.sub(emoji_pattern, '', text)
        #text = re.sub(r'[^a-zA-Z\s]', '', text)
        text = re.sub(r'(?<!\w)(?<!\.)[^\w\s]*', '', text)
        return text

    def transform(self, X):

        transformed_X = X.copy()

        transformed_X['description'] = transformed_X['description'].str.lower()
        transformed_X['description'] = transformed_X['description'].apply(self.clean_data)

        return transformed_X["description"].values

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X)

    def spliting(self, data):
        All_sentences = []
        for text in data:
            sentences = []
            for sentence in text.split("."):
                if len(sentence) > 1:
                    sentences.append(sentence.strip())
            All_sentences.extend(sentences[0:-1])
        return All_sentences


In [ ]:
class SBERT_Embedder:
    def __init__(self, model_name='all-MiniLM-L6-v2'):
        """
        Initialize the SBERT embedder with the specified model.
        """
        self.embedder = SentenceTransformer(model_name)
        self.vector_size = self.embedder.get_sentence_embedding_dimension()
        logging.info(f"Model '{model_name}' loaded with vector size {self.vector_size}.")

    @staticmethod
    def preprocess_text(text):
        """
        Normalize and preprocess the text.
        """
        text = text.lower()
        text = text.translate(str.maketrans('', '', string.punctuation))
        return text

    def create_embedding_for_document(self, document):
        """
        Create an embedding for a given document using the embedding model.
        """
        document = self.preprocess_text(document)
        embedding = self.embedder.encode(document).tolist()
        logging.info(f"Embedding created for document: {document}")
        return embedding

    def create_embeddings_for_documents(self, documents, batch_size=32):
        """
        Create embeddings for a list of documents, where each document is a list of sentences.
        """
        # Concatenate sentences for each document
        concatenated_documents = [' '.join([self.preprocess_text(sentence) for sentence in doc]) for doc in documents]
        embeddings = self.embedder.encode(concatenated_documents, batch_size=batch_size).tolist()
        logging.info(f"Embeddings created for {len(documents)} documents.")
        return embeddings

    def get_query_vector(self, query):
        """
        Convert a query into a vector using the embedding model.
        """
        query = self.preprocess_text(query)
        query_vector = self.embedder.encode(query).tolist()
        logging.info(f"Query vector created for query: {query}")
        return query_vector

In [ ]:
class BGE_Embedder:
    def __init__(self, model_name="BAAI/bge-reranker-v2-m3", vector_size=1024):
        """
        Initialize the BGE embedder with the specified model.
        """
        print("Uploading model")
        self.embedder = FlagModel(model_name, use_fp16=True)
        self.vector_size = vector_size
        logging.info(f"Model '{model_name}' loaded with vector size {self.vector_size}.")

    @staticmethod
    def preprocess_text(text):
        """
        Normalize and preprocess the text.
        """
        text = text.lower()
        text = text.translate(str.maketrans('', '', string.punctuation))
        return text

    def create_embedding_for_document(self, document):
        """
        Create an embedding for a given document using the embedding model.
        """
        document = self.preprocess_text(document)
        embedding = self.embedder.encode(document)
        logging.info(f"Embedding created for document: {document}")
        return embedding

    def create_embeddings_for_documents(self, documents, batch_size=32):
        """
        Create embeddings for a list of documents, where each document is a list of sentences.
        """
        # Concatenate sentences for each document
        concatenated_documents = [' '.join([self.preprocess_text(sentence) for sentence in doc]) for doc in documents]
        embeddings = self.embedder.encode(concatenated_documents, batch_size=batch_size).tolist()
        logging.info(f"Embeddings created for {len(documents)} documents.")
        return embeddings

    def get_query_vector(self, query):
        """
        Convert a query into a vector using the embedding model.
        """
        query = self.preprocess_text(query)
        query_vector = self.embedder.encode(query).tolist()
        logging.info(f"Query vector created for query: {query}")
        return query_vector

In [ ]:
class Qdrant_VB:
    def __init__(self, api_key, qdrant_url, collection_name, vector_size):
        self.api_key = api_key
        self.qdrant_url = qdrant_url
        self.collection_name = collection_name
        self.qdrant_client = QdrantClient(url=qdrant_url, api_key=api_key)
        self.vector_size = vector_size

    def create_qdrant_collection_if_not_exist(self):
        """
        Create a Qdrant collection if it does not already exist.
        """
        try:
            if not self.qdrant_client.collection_exists(self.collection_name):
                self.qdrant_client.create_collection(
                    collection_name=self.collection_name,
                    vectors_config=VectorParams(size=self.vector_size, distance=Distance.COSINE)
                )
                logging.info(f"Collection '{self.collection_name}' created.")
            else:
                logging.info(f"Collection '{self.collection_name}' already exists.")
        except Exception as e:
            logging.error(f"Error creating collection '{self.collection_name}': {e}")

    def add_embedding_to_qdrant(self, embedding_vector, document, doc_id):
        """
        Add an embedding vector and corresponding document information to a Qdrant collection.
        """
        try:
            self.qdrant_client.upsert(
                collection_name=self.collection_name,
                points=[PointStruct(
                    id=doc_id,
                    vector=embedding_vector,
                    payload={"text": document}
                )]
            )
            logging.info(f"Document ID {doc_id} added to collection '{self.collection_name}'.")
        except Exception as e:
            logging.error(f"Failed to add document ID {doc_id} to collection '{self.collection_name}': {e}")

    def search_similar_vectors(self, query_vector, limit=5):
        """
        Search for vectors in Qdrant that are similar to the query vector.
        """
        try:
            return self.qdrant_client.search(
                collection_name=self.collection_name, query_vector=query_vector, limit=limit
            )
        except Exception as e:
            logging.error(f"Error searching for similar vectors: {e}")
            return []

    def retrieve_text_by_ids(self, ids):
        """
        Retrieve the text of documents from Qdrant given their IDs.
        """
        try:
            results = self.qdrant_client.retrieve(collection_name=self.collection_name, ids=ids)
            return [result.payload["text"] for result in results]
        except Exception as e:
            logging.error(f"Error retrieving texts by IDs: {e}")
            return []

In [ ]:
class Annoy:
    def __init__(self, vector_size, metric='angular'):
        """
        Initialize the Annoy index with the specified vector size and distance metric.
        """
        self.vector_size = vector_size
        self.metric = metric
        self.annoy_index = AnnoyIndex(self.vector_size, self.metric)

    def add_embeddings(self, embeddings):
        """
        Add embeddings to the Annoy index.
        """
        if not isinstance(embeddings, np.ndarray) or embeddings.ndim != 2:
            raise ValueError("embeddings must be a 2D NumPy array")

        for i, vec in enumerate(embeddings):
            self.annoy_index.add_item(i, vec)

    def build_index(self, num_trees=10):
        """
        Build the Annoy index with the specified number of trees.
        """
        if not isinstance(num_trees, int) or num_trees <= 0:
            raise ValueError("num_trees must be a positive integer")

        self.annoy_index.build(num_trees)

    def save_index(self, index_path='annoy_index.ann'):
        """
        Save the Annoy index to disk.
        """
        if not isinstance(index_path, str):
            raise ValueError("index_path must be a string")

        try:
            self.annoy_index.save(index_path)
            print(f"Index saved to {index_path}")
        except Exception as e:
            print(f"An error occurred while saving the index: {e}")

    def load_index(self, index_path):
        """
        Load the Annoy index from disk.
        """
        if not isinstance(index_path, str):
            raise ValueError("index_path must be a string")

        try:
            self.annoy_index.load(index_path)
            print(f"Index loaded from {index_path}")
        except Exception as e:
            print(f"An error occurred while loading the index: {e}")

    def query(self, vector, n=10):
        """
        Query the Annoy index to find the nearest neighbors of the given vector.

        Parameters:
        vector (list): The query vector.
        n (int, optional): The number of nearest neighbors to return. Default is 10.

        Returns:
        list: The list of indices of the nearest neighbors.
        """
        if not isinstance(vector, list):
            raise ValueError("vector must be a list")

        return self.annoy_index.get_nns_by_vector(vector, n)


In [ ]:
import faiss

class FaissIndexer:
    """
    A class to create and manage a FAISS index for efficient similarity search.

    Attributes:
        dimension (int): The dimensionality of the vectors to be indexed.
        index_type (str): The type of FAISS index to use ('Flat', 'IVFFlat', 'PQ').
        nlist (int): The number of clusters to use for the 'IVFFlat' index type.
        m (int): The number of subquantizers to use for the 'PQ' index type.
        index (faiss.Index): The FAISS index object.

    Methods:
        create_index(): Creates and returns the FAISS index based on the specified type.
        train(data): Trains the FAISS index with the provided data if the index type requires training.
        add(data): Adds the provided data to the FAISS index.
        search(query_vector, num_neighbors): Searches the FAISS index for the nearest neighbors of the query vector.
    """

    def __init__(self, dimension, index_type='IVFFlat', nlist=100, m=8):
        """
        Initializes the FaissIndexer with the given parameters.

        Parameters:
            dimension (int): The dimensionality of the vectors to be indexed.
            index_type (str, optional): The type of FAISS index to use. Default is 'IVFFlat'.
            nlist (int, optional): The number of clusters to use for the 'IVFFlat' index type. Default is 100.
            m (int, optional): The number of subquantizers to use for the 'PQ' index type. Default is 8.
        """
        self.dimension = dimension
        self.index_type = index_type
        self.nlist = nlist
        self.m = m
        self.index = self.create_index()

    def create_index(self):
        """
        Creates and returns the FAISS index based on the specified type.

        Returns:
            faiss.Index: The created FAISS index object.

        Raises:
            ValueError: If an unknown index type is specified.
        """
        if self.index_type == 'Flat':
            return faiss.IndexFlatL2(self.dimension)
        elif self.index_type == 'IVFFlat':
            quantizer = faiss.IndexFlatL2(self.dimension)
            return faiss.IndexIVFFlat(quantizer, self.dimension, self.nlist)
        elif self.index_type == 'PQ':
            return faiss.IndexPQ(self.dimension, self.m, 8)
        else:
            raise ValueError(f"Unknown index type: {self.index_type}")

    def train(self, data):
        """
        Trains the FAISS index with the provided data if the index type requires training.

        Parameters:
            data (np.ndarray): The training data to be used for training the index.
        """
        if hasattr(self.index, 'train'):
            self.index.train(data)

    def add(self, data):
        """
        Adds the provided data to the FAISS index.

        Parameters:
            data (np.ndarray): The data to be added to the index.
        """
        self.index.add(data)

    def search(self, query_vector, num_neighbors):
        """
        Searches the FAISS index for the nearest neighbors of the query vector.

        Parameters:
            query_vector (np.ndarray): The query vector to search for.
            num_neighbors (int): The number of nearest neighbors to return.

        Returns:
            np.ndarray: The indices of the nearest neighbors.
        """
        distances, indices = self.index.search(query_vector.reshape(1, -1), num_neighbors)
        return indices[0]


In [ ]:
# initiate chuncker
custom_transformer = Chunking()
df = custom_transformer.read_data("data.csv")
transformed_df = custom_transformer.fit_transform(df)
documents = custom_transformer.spliting(transformed_df)

print("creating Embedderr and Qdrant object ")

### usage
api_key = "mt0L253URDvlCcmxCTEzmbjuHnKlFpI9zmB3URtsMwpd6OUuk2aM3Q"
qdrant_url = "https://9620ee82-03f9-4f04-a642-b006b25e2c7c.us-east4-0.gcp.cloud.qdrant.io"
qdrant_object = Qdrant_VB(api_key, qdrant_url, "new_collection", 1024)
print("Qdrant object created")
bge = BGE_Embedder()
print("BGE created")
counter = 0
# for doc in documents:
#     counter += 1
#     qdrant_object.create_qdrant_collection_if_not_exist()
#     document_embedding = bge.create_embeddings_for_documents(doc)
#     qdrant_object.add_embedding_to_qdrant(document_embedding, doc, counter)
#     print("doc", doc, " added successfully")

creating Embedderr and Qdrant object 
Qdrant object created
Uploading model


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of XLMRobertaModel were not initialized from the model checkpoint at BAAI/bge-reranker-v2-m3 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BGE created


In [ ]:

def create_and_save_embeddings(documents):
    """
    Creates and saves embeddings for the provided documents using SBERT and BGE.

    Parameters:
        documents (list of str): List of documents to create embeddings for.

    Returns:
        tuple: A tuple containing the SBERT and BGE embedders.
    """
    sbert_embedder = SBERT_Embedder()
    sbert_embeddings = sbert_embedder.create_embeddings_for_documents(documents)
    sbert_embeddings_np = np.array(sbert_embeddings)
    with open('SBERTData.pickle', 'wb') as f:
        pickle.dump(sbert_embeddings_np, f)

    bge_embedder = BGE_Embedder()
    # data = []
    # for document in documents:
    #     bge_embeddings = bge_embedder.create_embedding_for_document(document)
    #     data.append(bge_embeddings)
    # bge_embeddings_np = np.array(data)
    # with open('BGEData.pickle', 'wb') as f:
    #     pickle.dump(bge_embeddings_np, f)
    return sbert_embedder, bge_embedder

def load_embeddings_based_on_choice(embedder_type):
    """
    Loads embeddings from a file based on the selected embedder type.

    Parameters:
        embedder_type (str): The type of embedder ('SBERT' or 'BGE').
        embedder: The embedder object used to get the vector size.

    Returns:
        tuple: A tuple containing the document embeddings as a NumPy array and the vector size.

    Raises:
        ValueError: If an unsupported embedder type is specified.
    """
    if embedder_type == "SBERT":
        file_path = 'SBERTData.pickle'
        embedder = SBERT_Embedder()
        vector_size = embedder.vector_size
        with open(file_path, 'rb') as f:
            document_embeddings_np = pickle.load(f)
    elif embedder_type == "BGE":
        file_path = 'BGEData.pickle'
        embedder = BGE_Embedder()
        vector_size = embedder.vector_size
        with open(file_path, 'rb') as f:
            document_embeddings_np = pickle.load(f)
    else:
        raise ValueError(f"Unsupported embedder type: {embedder_type}")
    return document_embeddings_np, vector_size,embedder

def create_index(query, embedder, indextype, document_embeddings_np, vector_size):
    """
    Creates an index and searches for the nearest neighbors of a query.

    Parameters:
        query (str): The query string.
        embedder: The embedder object used to create the query vector.
        indextype (str): The type of index to use ('Annoy' or 'Faiss').
        document_embeddings_np (np.ndarray): The document embeddings.
        vector_size (int): The size of the embedding vectors.

    Returns:
        list: The indices of the nearest neighbors.

    Raises:
        ValueError: If an unsupported index type is specified.
    """
    if indextype == "ANNOY":
        Aindex = Annoy(vector_size)
        Aindex.add_embeddings(document_embeddings_np)
        Aindex.build_index(num_trees=100)
        Aindex.save_index(index_path='annoy_index.ann')
        query_vector = embedder.get_query_vector(query)
        neighbors = Aindex.query(query_vector, n=10)
    elif indextype == "FAISS":
        Findex = FaissIndexer(vector_size)
        Findex.create_index()
        Findex.train(document_embeddings_np)
        Findex.add(document_embeddings_np)
        query_vector = np.array(embedder.get_query_vector(query))
        neighbors = Findex.search(query_vector, num_neighbors=10)
    else:
        raise ValueError(f"Unsupported index type: {indextype}")
    return neighbors


In [ ]:
sbert_embedder,bge_embedder=create_and_save_embeddings(documents)

In [ ]:
document_embeddings_np,vector_size,embedder=load_embeddings_based_on_choice("BGE")
neighbors=create_index("Learn how to design large-scale systems", embedder,"FAISS",document_embeddings_np,vector_size)

Uploading model


Some weights of XLMRobertaModel were not initialized from the model checkpoint at BAAI/bge-reranker-v2-m3 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print("\nNearest Neighbors:")
for neighbor in neighbors:
    print(f"Document {neighbor + 1}: {documents[neighbor]}")


Nearest Neighbors:
Document 1: learn how to design large-scale systems
Document 1218: build forms from json schema
Document 474: explorations in core
Document 90: resume template for chinese program
Document 181: resume template for chinese program
Document 1086: provides dynamic roles based authorisation for node
Document 2073: a swipeable cards interface
Document 1530: implementation of efficientnet model
Document 2264: a javascript framework building upon but not dependant on) prototype's class
Document 1866: and adjust the sales solution system module 1


########################################################################################################################################

In [ ]:
["learn how to design large-scale systems",
"build forms from json schema",
"explorations in core",
"resume template for chinese program",
"resume template for chinese program",
"provides dynamic roles based authorisation for node",
"a swipeable cards interface",
" implementation of efficientnet model",
"a javascript framework building upon but not dependant on) prototype's class",
" and adjust the sales solution system module 1"]

In [ ]:
!pip install Flask


In [ ]:
!pip install flask ngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 13.0 MB/s eta 0:00:00


In [ ]:
os.makedirs('templates', exist_ok=True)


In [ ]:
from flask import Flask
from flask import request
from flask_ngrok import run_with_ngrok

In [ ]:
from flask import Flask, render_template, request

app = Flask(__name__)
@app.route('/')
def index():
    return render_template('search.html')

@app.route('/search', methods=['POST'])
def search():
    query = request.form.get('query')
    embedding_type = request.form.get('embedding')
    similarity_method = request.form.get('similarity')

    # Replace these functions with your actual implementations
    document_embeddings_np, vector_size, embedder = load_embeddings_based_on_choice(embedding_type)
    res = create_index(query, embedder, similarity_method, document_embeddings_np, vector_size)

    # # Debug print to see received values
    # print(f"Query: {query}, Embedding: {embedding_type}, Similarity: {similarity_method}")
    # print(f"Results: {results}")
    return render_template('result.html', results=res)
if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
